In [ ]:
import nrpy.reference_metric as refmetric
import nrpy.c_function as cfc
import nrpy.indexedexp as ixp
import nrpy.helpers.jacobians as jac
import nrpy.c_codegen as ccg  # NRPy+: C code generation
import sympy as sp
import nrpy.infrastructures.BHaH.general_relativity.ADM_Initial_Data_Reader__BSSN_Converter as IDConvert

In [ ]:
body = IDConvert.Cfunction_ADM_SphorCart_to_Cart(include_T4UU=True)
cfc.register_CFunction(desc="Sph_to_Cart", CoordSystem_for_wrapper_func="Spherical", name="Sph_to_Cart", body=body)
f = cfc.CFunction_dict['Sph_to_Cart__rfm__Spherical'].generate_full_function()

In [ ]:
# print(body)

In [ ]:
def Cfunction_FUKA_ADM_Sph_to_Cart(
    IDCoordSystem: str = "Spherical",
    include_T4UU: bool = False,
) -> str:
    """
    Convert ADM variables from FUKA in Spherical Basis to the Cartesian basis.
    Note: It's intended use is a lambda rather than a function call

    :param IDCoordSystem: The input coordinate system. Defaults to "Spherical".
    :param include_T4UU: Whether to include the stress-energy tensor. Defaults to False.

    :return: The name of the generated C function.
    """
    desc = "Convert ADM variables from the spherical or Cartesian basis to the Cartesian basis"
    c_type = "static void"
    name = "ADM_Spherical_to_Cart"
    params = ""
    
    # Set reference_metric to the IDCoordSystem
    rfm = refmetric.reference_metric[IDCoordSystem]
    body = 'const double xCart[3] = {x, y, z};'
    body += rf"""
      // Perform the basis transform on ADM vectors/tensors from {IDCoordSystem} to Cartesian:

      // Set destination xx[3] based on desired xCart[3]
      double xx0,xx1,xx2;
      """ + ccg.c_codegen(
            rfm.Cart_to_xx,
            ["xx0", "xx1", "xx2"],
            include_braces=True,
        ).replace(
            "Cartx", "xCart[0]"
        ).replace(
            "Carty", "xCart[1]"
        ).replace(
            "Cartz", "xCart[2]"
        )

    body += r"""// Unpack initial_data for ADM vectors/tensors
"""
    body += f"const double N = quant_vals[ISO_VARS::ISO_ALPHA]; \n"
    body += f"double const omega = quant_vals[ISO_VARS::ISO_OMEGA]; \n"
    body += f"const double Omega = (*bconfig)(BCO_PARAMS::OMEGA); \n"
    body += f"const double U_factor = (Omega - omega) / N * xx0 * sin(xx2); \n"    
    
    body += f"const double domega_dr = quant_vals[ISO_VARS::ISO_DOMEGA_DR]; \n"
    body += f"const double domega_dt = quant_vals[ISO_VARS::ISO_DOMEGA_DTHETA]; \n"
    body += "\n"
    
    body += f"const double FluidVelU0 = 0.0; // r\n"
    body += f"const double FluidVelU1 = 0.0; // theta\n"
    body += f"const double FluidVelU2 = U_factor; // phi\n"
    
    body += f"const double betaSphericalU0 = 0.0; // r\n"
    body += f"const double betaSphericalU1 = 0.0; // theta\n"
    body += f"const double betaSphericalU2 = - quant_vals[ISO_VARS::ISO_OMEGA]; // phi\n"
    body += "\n"
    
    body += f"const double A = quant_vals[ISO_VARS::ISO_METRIC_A];\n"
    body += f"const double B = quant_vals[ISO_VARS::ISO_METRIC_B];\n"
    body += "\n"
    
    for j in range(3):
        for k in range(j, 3):
            if not j == k:
                body += f"const double gammaSphericalDD{j}{k} = 0.0;\n"
    body += f"const double gammaSphericalDD00 = A * A;\n"
    body += f"const double gammaSphericalDD11 = A * A * xx0 * xx0;\n"
    body += f"const double gammaSphericalDD22 = B * B * xx0 * xx0 * sin(xx2) * sin(xx2) ;\n"
    body += "\n"
    for j in range(3):
        for k in range(j, 3):
            if not (j == 0 and k == 2) and not (j == 1 and k == 2) :
                body += f"const double KSphericalDD{j}{k} = 0.0;\n"
    body += f"const double KSphericalDD02 = - gammaSphericalDD22 / 2.0 / N * domega_dr;\n"
    body += f"const double KSphericalDD12 = - gammaSphericalDD22 / 2.0 / N * domega_dt;\n"
    # Read stress-energy tensor in spherical or Cartesian basis if desired.
    # if include_T4UU:
    #     for mu in range(4):
    #         for nu in range(mu, 4):
    #             body += f"const double T4SphorCartUU{mu}{nu} = initial_data->T4SphorCartUU{mu}{nu};\n"
    #     body += "\n"



    # Define the input variables:
    gammaSphericalDD = ixp.declarerank2("gammaSphericalDD", symmetry="sym01")
    KSphericalDD = ixp.declarerank2("KSphericalDD", symmetry="sym01")
    FluidUSphericalU = ixp.declarerank1("FluidVelU")
    betaSphorCartU = ixp.declarerank1("betaSphericalU")
    # T4SphorCartUU = ixp.declarerank2("T4SphorCartUU", symmetry="sym01", dimension=4)

    # Compute Jacobian to convert to Cartesian coordinates
    gammaCartDD = jac.basis_transform_tensorDD_from_rfmbasis_to_Cartesian(
        IDCoordSystem, gammaSphericalDD
    )
    KCartDD = jac.basis_transform_tensorDD_from_rfmbasis_to_Cartesian(
        IDCoordSystem, KSphericalDD
    )
    FluidUCartU = jac.basis_transform_vectorU_from_rfmbasis_to_Cartesian(
        IDCoordSystem, FluidUSphericalU
    )
    betaCartU = jac.basis_transform_vectorU_from_rfmbasis_to_Cartesian(
        IDCoordSystem, betaSphorCartU
    )
    # T4CartUU = cast(Sequence[Sequence[sp.Expr]], ixp.zerorank2(dimension=4))
    # if include_T4UU:
    #     T4CartUU = jac.basis_transform_4tensorUU_from_time_indep_rfmbasis_to_Cartesian(
    #         IDCoordSystem, T4SphorCartUU
    #     )
    CART_COORD = ['X', 'Y', 'Z']
    # alpha = sp.symbols("quant_vals[ISO_VARS::ISO_ALPHA]", real=True)
    list_of_output_exprs = []
    list_of_output_varnames = []
    for i in range(3):
        list_of_output_exprs += [FluidUCartU[i]]
        list_of_output_varnames += [f"out_pw[OUTPUT_VARS::VEL{CART_COORD[i]}]"]
        list_of_output_exprs += [betaCartU[i]]
        list_of_output_varnames += [f"out_pw[OUTPUT_VARS::BETA{CART_COORD[i]}]"]
        for j in range(i, 3):
            list_of_output_exprs += [gammaCartDD[i][j]]
            list_of_output_varnames += [f"out_pw[OUTPUT_VARS::G{CART_COORD[i]}{CART_COORD[j]}]"]
            list_of_output_exprs += [KCartDD[i][j]]
            list_of_output_varnames += [f"out_pw[OUTPUT_VARS::K{CART_COORD[i]}{CART_COORD[j]}]"]
    # if include_T4UU:
    #     for mu in range(4):
    #         for nu in range(mu, 4):
    #             list_of_output_exprs += [T4CartUU[mu][nu]]
    #             list_of_output_varnames += [f"ADM_Cart_basis->T4UU{mu}{nu}"]

    # Sort the outputs before calling outputC()
    # https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w
    list_of_output_varnames, list_of_output_exprs = (
        list(t)
        for t in zip(*sorted(zip(list_of_output_varnames, list_of_output_exprs)))
    )

    body += ccg.c_codegen(
        list_of_output_exprs,
        list_of_output_varnames,
        verbose=False,
        include_braces=False,
    )

    return cfc.CFunction(
        subdirectory=IDCoordSystem,  # Probably not needed
        desc=desc,
        c_type=c_type,
        name=name,
        params=params,
        include_CodeParameters_h=False,
        body=body,
    ).full_function

In [ ]:
a = Cfunction_FUKA_ADM_Sph_to_Cart()

In [ ]:
print(a)